# Contrôle des joints du bras Kinova

Ce script Python permet de contrôler les trois premiers degrés de liberté du bras robotique Gen3 Lite en envoyant des commandes conjointes. Il fonctionne autant avec le bras simulé que le bras réel.

### Le numéro de votre équipe

In [1]:
# Insérez ici le numéro de votre équipe
VOTRE_NUMERO_EQUIPE = 0

### Import des paquets Python

In [2]:
import sys; sys.path.append('/opt/ros/noetic/lib/python3/dist-packages')
sys.path.append('/usr/lib/python3/dist-packages')
sys.path.append('/home/admin_mec/mobile_manip_ws/devel/lib/python3/dist-packages/')
from os import environ
environ['ROS_MASTER_URI'] = "http://localhost:1137{}/".format(VOTRE_NUMERO_EQUIPE)
import numpy as np
import rospy
import time
from time import sleep
import math
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
from mobile_manip.srv import ReachValues, GetValues, ReachName

###  Fonction pour envoyer des valeurs au robot simulé dans Gazebo

In [3]:
def send_joint_angles(ang):
    # Create the list of angles
    #rospy.wait_for_service('/mobile_manip/reach_joints', timeout=20)
    reach_joints_angles = rospy.ServiceProxy('/mobile_manip/reach_joints', ReachValues)
    return reach_joints_angles(ang)

### Fonction de simplification pour le projet 2

In [4]:
# Définissez les valeurs des 3 joints a controler en degrés
def dof3control(joint1, joint2, joint3):
    cmd = np.array([joint1, joint2, joint3, 0, 0, 0])
    send_joint_angles(cmd)

### Fonction pour la cinématique directe

In [5]:
# Accepte les valeurs theta, d, a, alpha. Les angles doivent être en radians.
def dh(theta,d,a,alpha):
    return np.array([
        [np.cos(theta), -1*np.sin(theta)*np.cos(alpha),    np.sin(theta)*np.sin(alpha), a*np.cos(theta)],
        [np.sin(theta),    np.cos(theta)*np.cos(alpha), -1*np.cos(theta)*np.sin(alpha), a*np.sin(theta)],
        [            0,                  np.sin(alpha),                  np.cos(alpha),               d],
        [            0,                              0,                              0,               1]
    ])

### Calculs de cinematique inverse

In [6]:
def ik(x,y,z):
    
    # Calcul des angles des joints
    px = x
    py = y
    pz = z
    
    joint1 = 0
    joint2 = 0
    joint3 = 0

    return np.array([joint1,joint2,joint3])

In [246]:
# Mouvement vers la position de départ.
dof3control(0,0,0)

In [7]:
# Execution des angles pour le point X de la trajectoire.

P1 = ik(0,0,0)/np.pi*180
print(P1)
dof3control(P1[0],P1[1],P1[2])

[ 30.017 -16.884 -43.535]
[-122.667  29.971 -13.583]
[-91.690  45.002 -58.540]


In [249]:
# Vérification de la pose finale
offsets_cart = [0.0, 0.0, 0.0]
rospy.wait_for_service('/mobile_manip/get_cartesian')
get_pose = rospy.ServiceProxy('/mobile_manip/get_cartesian', GetValues)
res = get_pose()
res_arr = np.array(res.val)
res_arr[:3] = (res_arr[:3]-offsets_cart)*1000.0
print(res_arr)

[ 3.054  678.348  379.766 -103.989 -0.099 -0.836]
